In [5]:
# packages
import pandas as pd
from mod02_build_bot_predictor import train_model

### Define a function to extract predictions from the model

In [6]:
def predict_bot(df, model=None):
    """
    Predict whether each account is a bot (1) or human (0).
    """
    if model is None:
        model = train_model()

    preds = model.predict(df)
    return pd.Series(preds, index=df.index)

### Define a function to evaluate model error

In [7]:
def confusion_matrix_and_metrics(y_true, y_pred):
    """
    Computes confusion matrix and common error rates for binary classification.

    Assumes labels:
      0 = negative class
      1 = positive class

    Returns:
      dict with:
        tn, fp, fn, tp
        misclassification_rate
        false_positive_rate
        false_negative_rate
    """
    tn = fp = fn = tp = 0

    for yt, yp in zip(y_true, y_pred):
        if yt == 0 and yp == 0:
            tn += 1
        elif yt == 0 and yp == 1:
            fp += 1
        elif yt == 1 and yp == 0:
            fn += 1
        elif yt == 1 and yp == 1:
            tp += 1
        else:
            raise ValueError("Labels must be 0 or 1")

    total = tn + fp + fn + tp

    misclassification_rate = (fp + fn) / total if total > 0 else 0.0
    false_positive_rate = fp / (fp + tn) if (fp + tn) > 0 else 0.0
    false_negative_rate = fn / (fn + tp) if (fn + tp) > 0 else 0.0

    return {
        "tp": tp,
        "tn": tn,
        "fp": fp,
        "fn": fn,
        "misclassification_rate": misclassification_rate,
        "false_positive_rate": false_positive_rate,
        "false_negative_rate": false_negative_rate,
    }


### Load the data

In [8]:
TRAIN_PATH = "mod02_data/train.csv"
train = pd.read_csv(TRAIN_PATH)

TEST_PATH = "mod02_data/test.csv"
test = pd.read_csv(TEST_PATH)

### Format the data by independent vs. dependent variables

In [9]:
X_train = train.drop(columns=["is_bot"])
y_train = train['is_bot']

X_test = test.drop(columns=["is_bot"])
y_test = test['is_bot']

### Build the model on training data

In [10]:
model = train_model(X_train, y_train)

### Get the model predictions on training and test data

In [11]:
y_pred_train = predict_bot(X_train, model)
y_pred_test = predict_bot(X_test, model)

### Check results on the training set (data used to build the model)

In [12]:
confusion_matrix_and_metrics(y_train, y_pred_train)

{'tp': 171,
 'tn': 2619,
 'fp': 18,
 'fn': 192,
 'misclassification_rate': 0.07,
 'false_positive_rate': 0.006825938566552901,
 'false_negative_rate': 0.5289256198347108}

### Check results on the test set (new data not yet seen by the model)

In [13]:
confusion_matrix_and_metrics(y_test, y_pred_test)

{'tp': 33,
 'tn': 846,
 'fp': 28,
 'fn': 93,
 'misclassification_rate': 0.121,
 'false_positive_rate': 0.032036613272311214,
 'false_negative_rate': 0.7380952380952381}

# Discussion Questions

### Based on the misclassification rate of your model, discuss your confidence in the ability to predict a bot. 

Based on the test-set misclassification rate of 12.1%, I am pretty confident in the model’s ability to predict whether an account is a bot. The model generalizes well from the training data to unseen test data, as indicated by the relatively small gap between training and test error. The model is not the best it could be, but it performs way better than random guessing and shows consistent predictive behavior on new data.

### What are potential ramifications of false positives from the model?

False positives occur when the model incorrectly labels a human account as a bot. The potential ramifications include unfairly restricting or penalizing legitimate users, damaging user trust in the platform, and possibly silencing authentic voices.

### What are potential ramifications of false negatives from the model?

False negatives occur when bot accounts are incorrectly classified as human. This can allow malicious or automated accounts to continue spreading spam, misinformation, or manipulative content.